Live Streaming & Sentiment Analysis

In [2]:
#Importing required libraries
#Tweepy installation required for streaming twitter data 
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener

#JSON library required for handling data returned from api calls
import json, time, sys

#NLTK library required for natural laguage processing
import nltk
from nltk import *
import random
from nltk.corpus import stopwords
from nltk.corpus import movie_reviews
from nltk.classify import ClassifierI

#URLLIB required for creating url's
import urllib

# Matplotlib required for creating live graph
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style

#In working directory create a file called reqimports.py
#Create twitter apps account and a new appliaction.
#Using twitter apps account populate access token, access key, customer secret and customer key in regimports file
#Register for a free language processing api key @ alchemyapi.com
#Put alchemy api key in alap variable of reqimports 
from reqimports import *

In [3]:
#Default variables to 0 
xaxis = 0
result = 0
pos=0
neg=0
neu=0
elm=0

style.use("ggplot")
#Create container for graph
fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)

In [4]:
#Sentiment Analyzer module
def sentanalyzer(t):
#Takes some text and encodes it so that it can be passed in a URL    
    passtext = urllib.quote(t)
#Hold apikey value in a variable    
    alapikey= alap
#Call alchemyapi to retrive sentiment of passed text    
    apicall = 'http://access.alchemyapi.com/calls/text/TextGetTextSentiment?text='+passtext+'&apikey='+alapikey+'&outputMode=json'
#Read alchemyapi output and store in a variable    
    json_string = urllib.urlopen(apicall).read()
#Parse json to retrieve variables of interest
    parsed_json = json.loads(json_string)
#If alchemy returns error then return some text, else return the type of sentiment of the text passed(positive/negative/neutral)    
    if (parsed_json['status'] == 'OK'):
        return parsed_json['docSentiment']['type']
    else:
        return 'eliminated'

In [5]:
#Input box to take input from user, this keyword will be used to filter tweets, only tweets containing this keyword would be streamed.
inputw = raw_input("Enter keyword to mine: ")

Enter keyword to mine: happy


In [6]:
#Creates a listener class
class listener(StreamListener):
#Function specifying things to do when data found    
    def on_data(self, data):
#Clean twitter data, get rid of symbols and fetch text only (actual data has so much more)         
        decoded = json.loads(data)
        cleaned_tweet = (decoded['text'].encode('ascii', 'ignore'))
#Pass cleaned tweet through sentiment analyzer module to get sentiment, store in sent_result variable
        sent_result = sentanalyzer(cleaned_tweet)
#Variables used in below calculation        
        global xaxis
        global result
        global pos
        global neg
        global neu
        global elm
#For every single tweet increment xaxis by 1 (meaning xaxis holds total number of tweets)        
        xaxis +=1
#Based on sentment increment or decrement result (+ for pos, - for neg, 0 for neutral or error)  
#Also store total number of pos, neg, neu, err tweets
        if(sent_result == 'positive'):
            result += 1
            pos += 1
        elif(sent_result == 'negative'):
            result -= 1
            neg += 1
        if(sent_result == 'neutral'):
            result += 0
            neu += 1
        elif(sent_result == 'eliminated'):
            result += 0
            elm += 1
#Incrementally store results in text file.            
        with open("tweet_sentiment_result.txt", "a") as tweet_results:
            tweet_results.write("%s,%s\n" % (xaxis, result))
        with open("tweet_sentiments.txt", "a") as tweet_sents:
            tweet_sents.write("%s,%s,%s,%s\n" % (pos, neg, neu, elm))    
        return True
#Function specifying things to do if listener errors out    
    def on_error(self, status):
#Just prints status        
        print status

In [7]:
#Create authentication handler
auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

In [ ]:
#Call listener
twitterStream = Stream(auth, listener())
#Stream tweets, filer by user specified keyword
twitterStream.filter(track=[inputw])

Naive Bayes Classifier

In [ ]:
# Create a list of all words in movie_reviews along with their categories (pos/neg)
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

In [ ]:
#Shuffle words and create a list with all words
random.shuffle(documents)
all_words = []
for w in movie_reviews.words():
    all_words.append(w.lower())
#Get count for all words
all_words = nltk.FreqDist(all_words)
#Create a list of top 3000 words
word_features = list(all_words.keys())[:3000]

In [ ]:
#Function to find features in passed sentence
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

In [ ]:
#Create set of features, a list of feature-category
featuresets = [(find_features(rev), category) for (rev, category) in documents]
#Divide feature set into training and test
training_set = featuresets[0:1900]
testing_set = featuresets[1900:]
#print(testing_set)
#Create classifier using NLTK's naive bayes classifier and movie_review corpus
classifier = nltk.NaiveBayesClassifier.train(training_set)
#print(classifier)
classifier.show_most_informative_features(15)
#Check acuracy of created classifier
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

In [ ]:
#Function to extract sentiment of passed sentence using naive bayes classifier trained using movie_reviews corpus
def sentiment(text):
    feats = find_features(text)
    #print(text)
    return classifier.classify(feats)

In [ ]:
#Manually checking if the naive bayes algorithm works as expected
sentiment("This is amazing, I totally recommend")

In [ ]:
#Manually checking if alchemy api gives correct sentiment for text passed
sentanalyzer("This is amazing, I totally recommend")